In [2]:
from pandas_plink import read_plink
import pandas as pd
import dask 
import dask.array as da
import dask_ml.preprocessing
import numpy as np

In [2]:
#(bim_test, fam_test, G_test) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-test_common')
#(bim_gwas, fam_gwas, G_gwas) = read_plink('/Users/jenniferblanc/polygenic_adaptation_stratification/output/Simulate_Genotypes/4PopSplit/S2/C1/genos-gwas_common')

In [3]:
(bim_test, fam_test, G_test) = read_plink('/Users/jenniferblanc/scratch/practice2')
(bim_gwas, fam_gwas, G_gwas) = read_plink('/Users/jenniferblanc/scratch/practice2')


Mapping files: 100%|██████████| 3/3 [00:00<00:00, 36.58it/s]


In [26]:
# Mean Center Test
X = G_test.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(X)
X = scaler.transform(X)

# Mean Center GWAS
M = G_gwas.transpose()
scaler = dask_ml.preprocessing.StandardScaler(with_mean=True, with_std=False).fit(M)
M = scaler.transform(M)


array([[ 0.01,  0.65,  0.02, ...,  0.06,  0.02,  0.01],
       [ 0.01,  0.65,  0.02, ...,  0.06,  0.02,  0.01],
       [ 0.01,  0.65,  0.02, ...,  0.06,  0.02,  0.01],
       ...,
       [ 0.01, -0.35,  0.02, ...,  0.06,  0.02,  0.01],
       [ 0.01, -0.35,  0.02, ...,  0.06,  0.02,  0.01],
       [ 0.01, -0.35,  0.02, ...,  0.06,  0.02,  0.01]])

In [30]:
# Calculate Test covariance matrix 
test_cov = dask.array.cov(X)

# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd(test_cov)
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
vals = dask.array.diag(1/s[0:(n-1)])
#print(vals.compute())
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose()).compute()

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)
Tm = Tm.compute()

print(s.compute())

[1.81307640e+00 7.40247160e-01 7.23882722e-01 1.55978577e-01
 1.52268980e-01 1.48400657e-01 1.39678984e-01 1.38267153e-01
 1.35395737e-01 1.33517740e-01 1.32123840e-01 1.29542960e-01
 1.28626007e-01 1.27031896e-01 1.23973820e-01 1.21614540e-01
 1.20881115e-01 1.18112240e-01 1.17607344e-01 1.16605537e-01
 1.15571335e-01 1.14425343e-01 1.12711056e-01 1.11322145e-01
 1.10204250e-01 1.09062465e-01 1.08369042e-01 1.07127559e-01
 1.05347683e-01 1.05323948e-01 1.03071717e-01 1.02810947e-01
 1.01730721e-01 9.97746646e-02 9.88898027e-02 9.84895872e-02
 9.67475916e-02 9.49590634e-02 9.42563098e-02 9.36683340e-02
 9.33362780e-02 9.20581869e-02 9.11812569e-02 8.97643196e-02
 8.94619146e-02 8.91625114e-02 8.83682125e-02 8.77454864e-02
 8.58734365e-02 8.56009587e-02 8.48164081e-02 8.43373614e-02
 8.37133641e-02 8.25394851e-02 8.12445572e-02 8.04286194e-02
 7.98176053e-02 7.96605214e-02 7.76019564e-02 7.71317253e-02
 7.65762008e-02 7.52522903e-02 7.46030242e-02 7.44001417e-02
 7.38422669e-02 7.226703

In [7]:
# Calculate Test covariance matrix 
test_cov = dask.array.cov(X)

# SVD of covariance matrix 
u, s, v = dask.array.linalg.svd_compressed(test_cov, test_cov.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
vals = dask.array.diag(1/s[0:(n-1)])
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

array([ 0.09999968,  0.1000137 ,  0.09999543,  0.09999992,  0.10000921,
        0.09999425,  0.09999653,  0.10001465,  0.10001292,  0.10000441,
        0.10000795,  0.10001284,  0.10001205,  0.09999748,  0.10000732,
        0.09999212,  0.10002119,  0.10000662,  0.10000992,  0.09998401,
        0.09997464,  0.10001134,  0.10001307,  0.09999827,  0.10002473,
        0.09999409,  0.09999488,  0.10000646,  0.09999984,  0.09998259,
        0.09999827,  0.09999732,  0.09998582,  0.10000693,  0.09998338,
        0.10001433,  0.10000756,  0.10000283,  0.10000638,  0.10000795,
        0.10000031,  0.10000725,  0.09998944,  0.1000026 ,  0.09999157,
        0.10001118,  0.09999787,  0.10000094,  0.09998913,  0.100013  ,
       -0.10000418, -0.10000087, -0.09999976, -0.10000441, -0.10000827,
       -0.10000496, -0.09999031, -0.10000425, -0.10000339, -0.1000089 ,
       -0.10001591, -0.0999978 , -0.10000701, -0.10000473, -0.10000835,
       -0.10000906, -0.09999764, -0.1000015 , -0.09999913, -0.10

In [8]:
# SVD of data matrix 
u, s, v = dask.array.linalg.svd_compressed(X, X.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
s = da.square(s) / (X.shape[1] -1)
print(s.compute())
vals = dask.array.diag(1/(s[0:(n-1)]))
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

[1.81311546e+00 7.40313421e-01 7.23885971e-01 1.55978584e-01
 1.52292375e-01 1.48436403e-01 1.39727369e-01 1.38267175e-01
 1.35400696e-01 1.33544938e-01 1.32125100e-01 1.29568133e-01
 1.28630927e-01 1.27032444e-01 1.23974504e-01 1.21623109e-01
 1.20881449e-01 1.18128346e-01 1.17627791e-01 1.16606292e-01
 1.15581636e-01 1.14425481e-01 1.12712649e-01 1.11326368e-01
 1.10208099e-01 1.09066118e-01 1.08373965e-01 1.07141679e-01
 1.05347864e-01 1.05330031e-01 1.03072270e-01 1.02846374e-01
 1.01736493e-01 9.97870128e-02 9.89327594e-02 9.85323567e-02
 9.67584649e-02 9.49590697e-02 9.42568550e-02 9.36721844e-02
 9.33509395e-02 9.20731916e-02 9.11853010e-02 8.97647924e-02
 8.94625044e-02 8.91627783e-02 8.83687932e-02 8.77495574e-02
 8.58927035e-02 8.56009792e-02 8.48179914e-02 8.43380685e-02
 8.37177370e-02 8.25395801e-02 8.12733850e-02 8.04292078e-02
 7.98203238e-02 7.96605620e-02 7.76029205e-02 7.71319489e-02
 7.65762065e-02 7.52805028e-02 7.46111702e-02 7.44083306e-02
 7.38440780e-02 7.227785

array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,
        0.1,  0.1,  0.1,  0.1,  0.1,  0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
       -0.1])

In [24]:
# SVD of covariance matrix 
u, s = dask.array.linalg.eig(X, X.shape[0])
n = s.shape[0]

# Make test vector
mat = np.zeros(n)
mat[0:int(n/2)] = 1
mat = mat - mat.mean(axis = 0)
smat = mat ** 2
sums = smat.sum(axis = 0)
stvec = mat / np.sqrt(sums)

# Make pseudoinverse 
s = da.square(s)
print(s.compute())
vals = dask.array.diag(1/(s[0:(n-1)]))
pinv = np.dot(u[0:n,0:(n-1)], vals).dot(u[0:n,0:(n-1)].transpose())

# Calculate Tm
K = np.dot(M, X.transpose()) / (M.shape[1] - 1)
Tm = np.dot(K,pinv).dot(stvec)

Tm.compute()

AttributeError: module 'dask.array.linalg' has no attribute 'eig'

In [22]:
#file_name="/Users/jenniferblanc/Test.txt"
#outF = open(file_name, "w")
#for i in range(Tm.shape[0]):
#    print(i)
#    outF.write(str(Tm[i].compute()))
#outF.close()

print(X)

dask.array<sub, shape=(100, 45361), dtype=float64, chunksize=(100, 1024), chunktype=numpy.ndarray>


In [9]:
Tvec = np.loadtxt("/Users/jenniferblanc/polygenic_adaptation_stratification/output/Calculate_Tm/4PopSplit/S2/Tvec.txt")

In [18]:
#da.store(Tm, /Users/jenniferblanc/scratch/testTm")
da.to_hdf5('/Users/jenniferblanc/scratch/testTm.hdf5', '/output', Tm)